## Saving data and Loading data

### Soft code for pruning neural network

In [1]:
import torch
import pandas as pd
from reducing_net import reduced_ann_net
from Net import Net, train_model, test_model
from utils import confusion, F1_score, loadDataset, saveNNParas
import time


feature_num = 11
hidden_num = 30
output_num = 3

load_net = Net(feature_num, hidden_num, output_num)
load_net.load_state_dict(torch.load('net_model_genre.pt'))
load_net.eval()

x_train, y_train = loadDataset('testing')
x_test, y_test = loadDataset('testing')


vectors = pd.read_excel('vector_angle_sample.xls', header=None)
print(vectors)

raw_df = pd.DataFrame({'row': vectors.iloc[:, 0],
                   'col': vectors.iloc[:, 1],
                   'vector': vectors.iloc[:, 2]})


   0   1         2
0  1   3  7.635018
1  2  13  7.812484
2  5  20  7.657552


In [4]:
increase_res = raw_df.sort_values('vector', ascending=True)
unique_row = increase_res.row.unique()
unique_col = increase_res.col.unique()

print(increase_res)
print(unique_row)
print(unique_col)


nets = []
times = []
old_net = load_net
cnt = 0
hidden_num = 30


   row  col    vector
0    1    3  7.635018
2    5   20  7.657552
1    2   13  7.812484
[1 5 2]
[ 3 20 13]


In [5]:
while cnt < 10:
    if len(unique_row) == 0 or len(unique_col) == 0:
        print("\n Finished! \n")
        break
        
    for index, row in increase_res.iterrows():
        if row['row'] in unique_row and row['col'] in unique_col and row['row'] not in unique_col:
            hidden_num -= 1
            new_net = reduced_ann_net(old_net, int(row['row']), int(row['col']), hidden_num)

            print("\n======= Net hidden size: {}==========\n".format(hidden_num))
            
            start_time = time.time()
            train_model(new_net, x_train, y_train, lr=0.03, epochs=100)
            stop_time = time.time()
            print("Retraining time: %s ms" % ((stop_time - start_time)*1000))
            
            start_time = time.time()
            acc, pred = test_model(new_net, x_test, y_test)
            stop_time = time.time()
            print("Execution time: %s ms" % ((stop_time - start_time)*1000))
            times.append((stop_time - start_time)*1000)

            mat = confusion(x_test.size(0), 3, pred, y_test)
            print("Confusion Matrix (after pruning)：")
            print(mat)
            F1_score(mat)

            nets.append(new_net)
            old_net = new_net

            saveNNParas(new_net, x_test, hidden_num)
            vectors = pd.read_excel('vector_angle.xls', header=None)
            if (vectors.empty):
                cnt = 10
                print("\n Finished: Vectors are empty! \n")
                break

            df = pd.DataFrame({'row': vectors.iloc[:, 0],
                           'col': vectors.iloc[:, 1],
                           'vector': vectors.iloc[:, 2]})

            increase_res = df.sort_values('vector', ascending=True)
            print(increase_res)

            unique_row = increase_res.row.unique()
            unique_col = increase_res.col.unique()
            
            cnt += 1
            break
            
print(times)


======= Net hidden size: 29==========



ValueError: can't optimize a non-leaf Tensor

In [ ]:
print(vectors)